<a href="https://colab.research.google.com/github/zalikvl25/github-slideshow/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Аринин бот

import nltk
import random
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split

In [19]:
with open('/content/BIG_CONFIG_.json', 'r') as f:
    BOT_CONFIG = json.load(f)

In [21]:

def match(text, example):#гибкая функция сравнения
    return nltk.edit_distance(text, example) / len(example) < 0.3 if len(example) >0 else False

def cleaner(text):#Функция чистки текста
    cleaned_text = ''
    for ch in text.lower():
        if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя abcdefghijklmnopqrstuvwxyz':
            cleaned_text = cleaned_text + ch
    return cleaned_text

def get_intent(text):#Функция определения интента текста
    for intent in BOT_CONFIG['intents']:
      if 'examples' in BOT_CONFIG['intents'][intent]:
        for example in BOT_CONFIG['intents'][intent]['examples']:
            if match(cleaner(text), cleaner(example)):
                return intent
        

In [22]:
X=[]
y=[]
for intent in BOT_CONFIG['intents']:
    if 'examples' in BOT_CONFIG['intents'][intent]:
        X += BOT_CONFIG['intents'][intent]['examples']
        y += [intent for i in range(len(BOT_CONFIG['intents'][intent]['examples']))]
#Создаем обучающую выборку для ML-модели

In [23]:
vectorizer = CountVectorizer(preprocessor=cleaner, ngram_range=(1,3), stop_words = ['а','и'])
#Создаем векторайзер - объект для превращения текста в вектора

In [24]:
vectorizer.fit(X)
X_vect = vectorizer.transform(X)
#Обучаем векторайзер на нашей выборке

In [25]:
X_train_vect, X_test_vect, y_train, y_test = train_test_split(X_vect, y, test_size = 0.3)
#Разбиваем выборку на train и на test

In [26]:
sgd = SGDClassifier() #Создаем модель
sgd.fit(X_vect, y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [27]:
sgd.score(X_vect, y) #Смотрим качество классификации

0.8978328173374613

In [29]:
def get_intent_by_model(text):#Функция, определяющая интент текста с помощью ML-модели
    return sgd.predict(vectorizer.transform([text]))[0]

In [30]:
def bot(text):#Функция бота
    intent=get_intent(text)    #1. Попытаться понять намерения сравнением по Левинштейну
    if intent is None:
        intent = get_intent_by_model(text)   #2. Понять намерения с помощью ML-модели
    return random.choice(BOT_CONFIG['intents'][intent]['responses']) 

In [ ]:
#question = ''
#while question not in ['выход', 'выключайся', 'хватит!']:
      #question=input()
      #print("Бот: " + bot(question))

In [ ]:
#!pip install python-telegram-bot --upgrade

In [35]:
import logging

from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context. Error handlers also receive the raised TelegramError object in error.
def start(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help_command(update: Update, _: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, _: CallbackContext) -> None:
    """Echo the user message."""

    update.message.reply_text(bot(update.message.text))


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("1796743771:AAGxgsPzCc2dT6GIsNJAnaN2fVzB8l4bQGg")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on noncommand i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [36]:
main()

2021-03-28 13:15:22,484 - apscheduler.scheduler - INFO - Scheduler started
2021-03-28 13:29:25,269 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
2021-03-28 13:29:25,275 - apscheduler.scheduler - INFO - Scheduler has been shut down
